In [2]:
import numpy as np

In [8]:
num_samples = 1000
num_arms = 4
mean = np.random.rand(num_arms,)
sigma = 1
sorted_mean = np.sort(mean)
Delta = sorted_mean[-1]-sorted_mean[-2]

In [9]:
class EE21B063_UCB:
  def __init__(self,num_arms,num_samples,sigma):
    self.__num_arms = num_arms
    self.__num_samples = num_samples
    self.__sigma = sigma
    self.pred_mean = np.zeros(num_arms,dtype=float)
    self.times_choosen = np.zeros(num_arms,dtype=float)
    self.time = 0
    self.init_over = 0
    self.prev_action = -1

  def get_action(self,reward):

    # Initialization pick all arms once
    if(self.init_over == 0):
      if(self.time != 0) : self.pred_mean[self.time-1] = reward
      self.times_choosen[self.time] = 1
      self.time = self.time + 1
      if(self.time == num_arms): self.init_over = 1
      print(self.time,self.pred_mean)
      return self.time - 1
    
    
    else:
      maxx = -10**20
      cur_action = -1
      # print(self.time, self.pred_mean )
      # Update
      if(self.prev_action == -1 ): self.pred_mean[self.time - 1] = reward
      else : self.pred_mean[self.prev_action] = ((self.times_choosen[self.prev_action] - 1.0)*self.pred_mean[self.prev_action] + reward)/(1.0*self.times_choosen[self.prev_action])
      ucb_list = []
      for i in range(num_arms):
         ucb = self.pred_mean[i] + sigma*np.sqrt(4.0*np.log(self.__num_samples+ 1))/(1.0*self.times_choosen[i])
         ucb_list.append(ucb)
         if(ucb > maxx):
            maxx = ucb
            cur_action = i
      # print(self.time,ucb_list,self.pred_mean)
      self.times_choosen[cur_action] =  1 + self.times_choosen[cur_action]
      self.time = self.time + 1
      self.prev_action = cur_action
      return cur_action
    


In [10]:
class Environment:
  def __init__(self,num_arms,mean,sigma,agent):
    self.__num_samples = num_samples
    self.__num_arms = num_arms
    self.__mean = mean
    self.__reward = -100
    self.__action = np.zeros((num_samples,),dtype = int)
    self.__agent = agent
    self.__regret = 0

  def run(self):
    for i in range(self.__num_samples):
      self.__action[i] = self.__agent.get_action(self.__reward)
      self.__reward = self.__mean[self.__action[i]]+sigma*np.random.randn()
    return self.__action


In [11]:
agent = EE21B063_UCB(num_arms,num_samples, sigma)
env = Environment(num_arms,mean,sigma,agent)
print(mean)
action_list = env.run()


[0.41758064 0.15942536 0.71661984 0.63020107]
1 [0. 0. 0. 0.]
2 [-1.07451565  0.          0.          0.        ]
3 [-1.07451565 -0.37361034  0.          0.        ]
4 [-1.07451565 -0.37361034  1.11191631  0.        ]


In [12]:
print(agent.pred_mean,agent.times_choosen)

[ 0.22223003 -0.21904834 -0.08869402  0.62603195] [ 19.   9.   9. 963.]
